# Intermediate component separation

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import pylab
pylab.rcParams['figure.figsize'] = 12, 16

import healpy as hp
import pysm

from fgbuster.observation_helpers import get_instrument, get_sky  # Predefined instrumental and sky-creation configurations
from fgbuster.visualization import corner_norm

# Imports needed for component separation
from fgbuster import (CMB, Dust, Synchrotron,  # sky-fitting model
                      basic_comp_sep)  # separation routine

## Input frequency maps
You have some frequency maps to clean, they can be either data or simulations.

### Simple case
Let's simulate a simple sky with `pysm`. ForeGroundBuster adds a couple of functions that make the process even easier.

In [ ]:
NSIDE = 64
sky_conf_simple = get_sky(NSIDE, 'c1d0s0') 
instrument = pysm.Instrument(get_instrument('cmbs4', NSIDE))
freq_maps_simple, noise = instrument.observe(pysm.Sky(sky_conf_simple), write_outputs=False)

We will focus on polarization-only component separation

In [ ]:
freq_maps_simple = freq_maps_simple[:, 1:]  # Select polarization

### Spatially varying spectral indices
Let's prepare also maps with spatially varying spectral indices. Similarly to the simple case above, we run the following (notice `d1s1`)

In [ ]:
NSIDE_PATCH = 8
sky_conf_vary = get_sky(NSIDE, 'c1d1s1')

We can still modify the sky configuration. In this case, we change the nside over which the spectral indices are allowed to vary

In [ ]:
for comp, param in [('dust', 'spectral_index'),
                    ('dust', 'temp'),
                    ('synchrotron', 'spectral_index')
                   ]:
    spectral_param = sky_conf_vary[comp][0][param]
    spectral_param[:] = hp.ud_grade(hp.ud_grade(spectral_param, NSIDE_PATCH),
                                    NSIDE)

Here is how they look like in constant and varying case. The rightmost plot shows the full resolution `pysm` template.

In [ ]:
comp = 'dust'
param = 'spectral_index'
hp.mollview(sky_conf_simple[comp][0][param], sub=(1,3,1), title='Constant index')
hp.mollview(sky_conf_vary[comp][0][param], sub=(1,3,2), title='Varying indices')
hp.mollview(get_sky(NSIDE, 'c1d1s1')[comp][0][param], sub=(1,3,3), title='Full resolution indices')

And now we generate the maps and select polarization

In [ ]:
freq_maps_vary, _ = instrument.observe(pysm.Sky(sky_conf_vary), write_outputs=False)
freq_maps_vary = freq_maps_vary[:, 1:] # Select polarization

## Component separation
The sky model we fit for is defined as a list of `Component` objects. They can be easily build from analytic SEDs, but for popular component types these are already implemented.

In [ ]:
components = [CMB(), Dust(353.), Synchrotron(23.)]

In [ ]:
# The starting point of the fit is the pysm default value, so let's shift it
components[1].defaults = [1.6, 22.]
components[2].defaults = [-2.7]

We are now ready to perform the component separation

In [ ]:
result = basic_comp_sep(components, instrument, freq_maps_simple,
                        options=dict(disp=True),  # verbose output
                        )

The input spectral parameters are recovered to numerical accuracy

In [ ]:
import numpy as np
inputs = [sky_conf_simple[comp][0][param][0]
          for comp, param in [('dust', 'spectral_index'),
                              ('dust', 'temp'),
                              ('synchrotron', 'spectral_index')]
         ]
print("%-20s\t%s\t%s" % ('', 'Estimated', 'Input'))
for param, val, ref in zip(result.params, result.x, inputs):
    print("%-20s\t%f\t%f" % (param, val, ref))

Their semi-analytic covariance is also provided, but remember that it is accurate only in the high signal-to-noise regime

In [ ]:
corner_norm(result.x, result.Sigma, labels=result.params, truths=inputs)

The amplitudes of the components are stacked in the `s` attribute and they are in the same format of the input frequency maps: Q and U healpix maps, in this case. Here is the U Stokes parameter for each of the components.

In [ ]:
hp.mollview(result.s[0,1], title='CMB', sub=(1,3,1))
hp.mollview(result.s[1,1], title='Dust', norm='hist', sub=(1,3,2))
hp.mollview(result.s[2,1], title='Synchrotron', norm='hist', sub=(1,3,3))

By taking the difference with the input template, we see that the error in the reconstruction is negligible.

In [ ]:
hp.mollview(result.s[1,1] 
            - sky_conf_simple['dust'][0]['A_U'] * pysm.convert_units('K_RJ', 'K_CMB', 353.),
            title='Dust', norm='hist', sub=(1,2,1))
hp.mollview(result.s[2,1] 
            - sky_conf_simple['synchrotron'][0]['A_U'] * pysm.convert_units('K_RJ', 'K_CMB', 23.), 
            title='Synchrotron', norm='hist', sub=(1,2,2))

## Component separation with varying indices
We now fit the spectral parameters independently over patches corresponding to healpix pixels with a given nside

In [ ]:
nside_fit = NSIDE_PATCH
result_vary = basic_comp_sep(components, instrument, freq_maps_vary, nside_fit)

As in the previous case, the amplitudes of the components are stacked in the `s`.

In [ ]:
hp.mollview(result_vary.s[0,1], title='CMB', sub=(1,3,1))
hp.mollview(result_vary.s[1,1], title='Dust', norm='hist', sub=(1,3,2))
hp.mollview(result_vary.s[2,1], title='Synchrotron', norm='hist', sub=(1,3,3))

When we take the difference with the input templates, the residuals are patchy. This is because the independent fit of the non-liner parameters has a different level of numerical accuracy for different patches. However, note that in all cases residuals are negligible: also this multi-patch cleaning has very high accuracy. 

In [ ]:
hp.mollview(result_vary.s[1,1] - 
            sky_conf_vary['dust'][0]['A_U'] * pysm.convert_units('K_RJ', 'K_CMB', 353.),
            title='Dust', norm='hist', sub=(1,2,1))
hp.mollview(result_vary.s[2,1] - 
            sky_conf_vary['synchrotron'][0]['A_U'] * pysm.convert_units('K_RJ', 'K_CMB', 23.), 
            title='Synchrotron', norm='hist', sub=(1,2,2))

The same is true for the non-linear parameters. Here are their reconstructed maps.

In [ ]:
for i, par in enumerate(result.params):
    hp.mollview(result_vary.x[i], title=par, sub=(1,3,i+1))

And here is the difference with the input templates.

In [ ]:
hp.mollview(hp.ud_grade(result_vary.x[0], NSIDE) - 
            sky_conf_vary['dust'][0]['spectral_index'],
            title=result.params[0], norm='hist', sub=(1,3,1))
hp.mollview(hp.ud_grade(result_vary.x[1], NSIDE) - 
            sky_conf_vary['dust'][0]['temp'],
            title=result.params[1], norm='hist', sub=(1,3,2))
hp.mollview(hp.ud_grade(result_vary.x[2], NSIDE) - 
            sky_conf_vary['synchrotron'][0]['spectral_index'],
            title=result.params[2], norm='hist', sub=(1,3,3))